In [61]:
import torch
from torchvision import transforms
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from PIL import Image
import numpy as np
import os

from sklearn.model_selection import train_test_split

# Converting Images to Tensor and Making train-test split

In [82]:
convert_tensor = transforms.ToTensor()
directory = 'no'
first = True

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        image = Image.open(f)
        tensor = convert_tensor(image).numpy()
        print(f,tensor.shape)
        if first:
            no_images = np.array([tensor])
            no_labels = np.array([[0]])
            first = False
        else:
            no_images = np.concatenate((no_images, np.array([tensor])))
            no_labels = np.concatenate((no_labels, np.array([[0]])))


no\1 no.jpeg
no\1 no.jpeg (1, 630, 630)
no\10 no.jpg
no\10 no.jpg (3, 201, 173)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 1 and the array at index 1 has size 3

In [73]:
convert_tensor = transforms.ToTensor()
directory = 'yes'
first = True

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        image = Image.
        tensor = convert_tensor(f).numpy()
        if first:
            print(tensor)
            yes_images = np.array([tensor[0]])
            yes_labels = np.array([[0]])
            first = False
        else:
            yes_images = np.concatenate((yes_images, np.array([tensor[0]])))
            yes_labels = np.concatenate((yes_labels, np.array([[0]])))

TypeError: pic should be PIL Image or ndarray. Got <class 'str'>

In [64]:
images = np.concatenate((no_images, yes_images))
labels = np.concatenate((no_labels, yes_labels))

In [66]:
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size = 0.2, random_state = 42)
train_images, test_images = train_images / 255.0, test_images / 255.0

# Modeling

In [68]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(1, 630, 630)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv2d. Consider increasing the input size. Received input shape [None, 1, 630, 630] which would produce output shape with a zero or negative value in a dimension.

In [ ]:
model.summary()